In [1]:
import torch
print(torch.cuda.is_available())
print(torch.version.cuda)

# conda install -c omgarcia gcc-6 # install GCC version 6
# conda install libgcc

True
11.1


In [2]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

from detectron2.config import get_cfg

In [3]:
from detectron2.data.datasets.coco import register_coco_instances, convert_to_coco_dict

from pathlib import Path

data_path = Path('data/SpermSegGS/')

if not Path('cocosplit.py').exists():
    !wget https://raw.githubusercontent.com/akarazniewicz/cocosplit/master/cocosplit.py
!python cocosplit.py --having-annotations -s 0.75 coco_train.json train.json val.json

register_coco_instances("sperm-train", {}, "train.json", "")
register_coco_instances("sperm-val", {}, "val.json", "")
register_coco_instances("sperm-test", {}, "coco_test.json", "")

Saved 33 entries in train.json and 12 in val.json


In [4]:
# augmentation ...

In [5]:
# https://dl.fbaipublicfiles.com/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl

In [6]:
from detectron2.engine import DefaultTrainer


import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], 'detr'))

from detr.d2.train_net import add_detr_config, Trainer


cfg = get_cfg()
add_detr_config(cfg)

cfg.merge_from_file('detrd2/configs/detr_segm_256_6_6_torchvision.yaml')
cfg.DATASETS.TRAIN = ("sperm-train",)
cfg.DATASETS.TEST = ("sperm-test",)
cfg.TEST.EVAL_PERIOD = 250
cfg.DATALOADER.NUM_WORKERS = 9
cfg.MODEL.WEIGHTS = 'https://dl.fbaipublicfiles.com/detr/detr-r50-dc5-f0fb7ef5.pth'
cfg.SOLVER.IMS_PER_BATCH = 1
# cfg.SOLVER.BASE_LR = 0.00020  # pick a good LR
# cfg.SOLVER.MAX_ITER = 2500    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset

# cfg.SOLVER.STEPS = []        # do not decay learning rate

# faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 32 
# # see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3  

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = Trainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

[07/04 04:49:47 d2.engine.defaults]: Model:
Detr(
  (detr): DETRsegm(
    (detr): DETR(
      (transformer): Transformer(
        (encoder): TransformerEncoder(
          (layers): ModuleList(
            (0): TransformerEncoderLayer(
              (self_attn): MultiheadAttention(
                (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
              )
              (linear1): Linear(in_features=256, out_features=2048, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
              (linear2): Linear(in_features=2048, out_features=256, bias=True)
              (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
              (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
              (dropout1): Dropout(p=0.1, inplace=False)
              (dropout2): Dropout(p=0.1, inplace=False)
            )
            (1): TransformerEncoderLayer(
              (self_attn): MultiheadAttention(
              

WARNING [07/04 04:49:47 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[07/04 04:49:47 d2.data.datasets.coco]: Loaded 33 images in COCO format from train.json
[07/04 04:49:47 d2.data.build]: Distribution of instances among all 3 categories:
|  category  | #instances   |  category  | #instances   |  category  | #instances   |
|:----------:|:-------------|:----------:|:-------------|:----------:|:-------------|
|    tail    | 90           |    mid     | 130          |    head    | 164          |
|            |              |            |              |            |              |
|   total    | 384          |            |              |            |              |
[07/04 04:49:47 d2.data.build]: Using training sampler TrainingSampler
[07/04 04:49:47 d2.data.common]: Serializing 33 elements to byte tensors and concatenating them all ...
[07/04 04:49:47 d2.data.common]: Serialized dataset takes 0.14 MiB


Some model parameters or buffers are not found in the checkpoint:
criterion.empty_weight
detr.bbox_attention.k_linear.{bias, weight}
detr.bbox_attention.q_linear.{bias, weight}
detr.detr.backbone.0.backbone.res2.0.conv1.norm.{bias, weight}
detr.detr.backbone.0.backbone.res2.0.conv1.weight
detr.detr.backbone.0.backbone.res2.0.conv2.norm.{bias, weight}
detr.detr.backbone.0.backbone.res2.0.conv2.weight
detr.detr.backbone.0.backbone.res2.0.conv3.norm.{bias, weight}
detr.detr.backbone.0.backbone.res2.0.conv3.weight
detr.detr.backbone.0.backbone.res2.0.shortcut.norm.{bias, weight}
detr.detr.backbone.0.backbone.res2.0.shortcut.weight
detr.detr.backbone.0.backbone.res2.1.conv1.norm.{bias, weight}
detr.detr.backbone.0.backbone.res2.1.conv1.weight
detr.detr.backbone.0.backbone.res2.1.conv2.norm.{bias, weight}
detr.detr.backbone.0.backbone.res2.1.conv2.weight
detr.detr.backbone.0.backbone.res2.1.conv3.norm.{bias, weight}
detr.detr.backbone.0.backbone.res2.1.conv3.weight
detr.detr.backbone.0.backb

The checkpoint state_dict contains keys that are not used by the model:
  transformer.encoder.layers.0.self_attn.{in_proj_bias, in_proj_weight}
  transformer.encoder.layers.0.self_attn.out_proj.{bias, weight}
  transformer.encoder.layers.0.linear1.{bias, weight}
  transformer.encoder.layers.0.linear2.{bias, weight}
  transformer.encoder.layers.0.norm1.{bias, weight}
  transformer.encoder.layers.0.norm2.{bias, weight}
  transformer.encoder.layers.1.self_attn.{in_proj_bias, in_proj_weight}
  transformer.encoder.layers.1.self_attn.out_proj.{bias, weight}
  transformer.encoder.layers.1.linear1.{bias, weight}
  transformer.encoder.layers.1.linear2.{bias, weight}
  transformer.encoder.layers.1.norm1.{bias, weight}
  transformer.encoder.layers.1.norm2.{bias, weight}
  transformer.encoder.layers.2.self_attn.{in_proj_bias, in_proj_weight}
  transformer.encoder.layers.2.self_attn.out_proj.{bias, weight}
  transformer.encoder.layers.2.linear1.{bias, weight}
  transformer.encoder.layers.2.linear2.

[07/04 04:49:48 d2.engine.train_loop]: Starting training from iteration 0


/home/step/.local/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/home/step/.local/lib/python3.9/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


ERROR [07/04 04:49:49 d2.engine.train_loop]: Exception during training:
Traceback (most recent call last):
  File "/home/step/Code/detectron2/detectron2/engine/train_loop.py", line 149, in train
    self.run_step()
  File "/home/step/Code/detectron2/detectron2/engine/defaults.py", line 497, in run_step
    self._trainer.run_step()
  File "/home/step/Code/detectron2/detectron2/engine/train_loop.py", line 273, in run_step
    loss_dict = self.model(data)
  File "/home/step/.local/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1051, in _call_impl
    return forward_call(*input, **kwargs)
  File "/home/step/Personal/UCH/2021-sem1/VisionComp/t4/detr/d2/detr/detr.py", line 177, in forward
    output = self.detr(images)
  File "/home/step/.local/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1051, in _call_impl
    return forward_call(*input, **kwargs)
  File "/home/step/Personal/UCH/2021-sem1/VisionComp/t4/detr/models/segmentation.py", line 58, in forward
    se

RuntimeError: CUDA out of memory. Tried to allocate 608.00 MiB (GPU 0; 7.79 GiB total capacity; 3.70 GiB already allocated; 651.44 MiB free; 4.45 GiB reserved in total by PyTorch)

In [ ]:
!ls /home/step/Personal/UCH/2021-sem1/VisionComp/t4/data/SpermSegGS/images/train/